# Data cleaning and feature engineering
___________________________________________________________________________________________

### Problem

Solar energy is a renewable energy source that depends on the irradiation data parameters to be efficient. Therefore, before investing in a new solar plant, it is necessary to gather the most solar data possible. 

### Acquisition of data 
1) Process a "raw" satellite image of the location.

2) Use a collection of past satellite images at that location to create a "background" 

3) Determine the cloud opacity - Decomposition of cloudy and cloud-free regions. Then, apply a 3D decomposition model to cloudy regions to characterize the thickness of each layer to sunlight.

4) Apply a modified version of the REST2 clear-sky radiation model, that allows us to use the latest global aerosol (dust, salt, smoke, etc.) and water vapor content to generate precise estimates of the solar radiation available to cloud-free regions.

### Purpose:

Using the data collected, we want to predict solar irradiance. 

For more information on the data acquisition process, please head over to https://solcast.com/solar-radiation-data/inputs-and-algorithms/!

#### Import Libraries

In [2]:
# Import Libraries
import numpy as np
import pandas as pd

#### Read in Dataframe

In [4]:
# Read in dataframe using pandas
df = pd.read_csv('../datasets/raw/solar_ir_abj_60M_ny.csv')

#### Dataframe column names

In [7]:
# Print the column names
print(df.columns)

# Print the number of columns
print(len(df.columns))

Index(['PeriodEnd', 'PeriodStart', 'Period', 'AirTemp', 'Azimuth',
       'CloudOpacity', 'DewpointTemp', 'Dhi', 'Dni', 'Ebh', 'Ghi',
       'GtiFixedTilt', 'GtiTracking', 'PrecipitableWater', 'RelativeHumidity',
       'SnowWater', 'SurfacePressure', 'WindDirection10m', 'WindSpeed10m',
       'Zenith', 'AlbedoDaily'],
      dtype='object')
21


We have 21 columns.

#### Feature selection

In [ ]:
# Feature selection
df = df[['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi',
       'PrecipitableWater', 'RelativeHumidity', 'SurfacePressure',
       'WindDirection10m', 'WindSpeed10m']]

Label each sample with their respective periods and important weather data (selected by a subject matter expert)

#### Randomly display 10 rows

In [8]:
# Randomly display 10 rows
df.sample(10)

,PeriodEnd,PeriodStart,Period,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,...,GtiFixedTilt,GtiTracking,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily
21815,2009-06-28T01:00:00Z,2009-06-28T00:00:00Z,PT60M,21.7,58,1.4,17.1,7,8,1,...,7,7,33.6,75.3,0.0,997.1,327,3.3,91,0.11
15995,2008-10-28T13:00:00Z,2008-10-28T12:00:00Z,PT60M,7.0,-119,75.4,5.7,26,0,0,...,24,26,19.4,91.4,0.0,986.7,360,4.9,79,0.13
119715,2020-08-28T05:00:00Z,2020-08-28T04:00:00Z,PT60M,26.9,8,60.4,18.7,0,0,0,...,0,0,36.0,60.8,0.0,1005.1,308,5.4,129,0.11
22319,2009-07-19T01:00:00Z,2009-07-19T00:00:00Z,PT60M,24.3,60,1.6,13.0,4,1,0,...,4,4,21.2,49.1,0.0,1002.6,259,3.4,92,0.11
80318,2016-02-29T16:00:00Z,2016-02-29T15:00:00Z,PT60M,12.2,-149,78.4,4.7,121,0,0,...,111,121,16.8,60.0,0.0,993.6,236,6.5,54,0.11
77444,2015-11-01T22:00:00Z,2015-11-01T21:00:00Z,PT60M,17.2,112,11.1,10.6,26,24,4,...,36,33,14.7,64.8,0.0,1002.8,243,4.8,87,0.10
99020,2018-04-18T22:00:00Z,2018-04-18T21:00:00Z,PT60M,11.4,96,0.0,0.4,98,641,251,...,338,806,11.3,46.8,0.0,998.2,264,4.8,67,0.10
8600,2007-12-25T10:00:00Z,2007-12-25T09:00:00Z,PT60M,1.3,-96,0.0,-4.3,0,0,0,...,0,0,8.3,66.4,0.0,1011.5,288,4.1,121,0.13
56620,2013-06-17T06:00:00Z,2013-06-17T05:00:00Z,PT60M,19.7,-9,26.1,17.0,0,0,0,...,0,0,23.2,84.3,0.0,1002.2,281,3.3,115,0.12
49862,2012-09-08T16:00:00Z,2012-09-08T15:00:00Z,PT60M,24.6,-147,26.8,22.1,377,209,159,...,558,563,38.9,86.0,0.0,997.5,185,7.2,40,0.10


#### Relabel column names

In [9]:
# Rename column names
old_names = ['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi','PrecipitableWater', 'RelativeHumidity',
             'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'] 
new_names = ['PeriodEnd','PeriodStart','Temperature', 'DHI', 'DNI', 'Radiation', 'Precipitation', 'Humidity', 
             'Pressure', 'WindDirection', 'WindSpeed']
df.rename(columns = dict(zip(old_names, new_names)), inplace = True)

#### Display dataframe

In [10]:
# Display dataframe
df.head()

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,GtiFixedTilt,GtiTracking,Precipitation,Humidity,SnowWater,Pressure,WindDirection,WindSpeed,Zenith,AlbedoDaily
0,2007-01-01T02:00:00Z,2007-01-01T01:00:00Z,PT60M,4.5,84,61.6,-1.0,0,0,0,...,0,0,22.1,67.3,0.0,1017.3,163,3.3,133,0.1
1,2007-01-01T03:00:00Z,2007-01-01T02:00:00Z,PT60M,4.6,72,61.8,-0.6,0,0,0,...,0,0,24.2,68.8,0.0,1016.7,160,3.8,144,0.1
2,2007-01-01T04:00:00Z,2007-01-01T03:00:00Z,PT60M,4.9,52,59.0,-0.2,0,0,0,...,0,0,25.5,69.4,0.0,1015.9,159,4.3,154,0.1
3,2007-01-01T05:00:00Z,2007-01-01T04:00:00Z,PT60M,5.4,20,76.9,0.2,0,0,0,...,0,0,26.2,69.2,0.0,1015.0,159,4.7,161,0.1
4,2007-01-01T06:00:00Z,2007-01-01T05:00:00Z,PT60M,5.8,-22,98.0,0.6,0,0,0,...,0,0,26.8,69.0,0.0,1014.2,159,5.0,161,0.1


Display a summary of the dataframe (show the datatype of each feature)

In [12]:
# Summary of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131421 entries, 0 to 131420
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   PeriodEnd      131421 non-null  object 
 1   PeriodStart    131421 non-null  object 
 2   Period         131421 non-null  object 
 3   Temperature    131421 non-null  float64
 4   Azimuth        131421 non-null  int64  
 5   CloudOpacity   131421 non-null  float64
 6   DewpointTemp   131421 non-null  float64
 7   DHI            131421 non-null  int64  
 8   DNI            131421 non-null  int64  
 9   Ebh            131421 non-null  int64  
 10  Radiation      131421 non-null  int64  
 11  GtiFixedTilt   131421 non-null  int64  
 12  GtiTracking    131421 non-null  int64  
 13  Precipitation  131421 non-null  float64
 14  Humidity       131421 non-null  float64
 15  SnowWater      131421 non-null  float64
 16  Pressure       131421 non-null  float64
 17  WindDirection  131421 non-nul

#### Convert data types for prediction

In [14]:
# Convert features to appropriate datatypes for predictions
df['Radiation']     = df['Radiation'].astype(float)
df['Temperature']   = df['Temperature'].astype(float) # or int
df['Pressure']      = df['Pressure'].astype(float)
df['Humidity']      = df['Humidity'].astype(int) # or int
df['WindDirection'] = df['WindDirection'].astype(float)
df['WindSpeed']     = df['WindSpeed'].astype(float)
df['PeriodStart'] = pd.to_datetime(df['PeriodStart']).dt.to_period('T').dt.to_timestamp()
df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd']).dt.to_period('T').dt.to_timestamp()

#### Display dataframe

In [15]:
# Summary of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131421 entries, 0 to 131420
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   PeriodEnd      131421 non-null  datetime64[ns]
 1   PeriodStart    131421 non-null  datetime64[ns]
 2   Period         131421 non-null  object        
 3   Temperature    131421 non-null  float64       
 4   Azimuth        131421 non-null  int64         
 5   CloudOpacity   131421 non-null  float64       
 6   DewpointTemp   131421 non-null  float64       
 7   DHI            131421 non-null  int64         
 8   DNI            131421 non-null  int64         
 9   Ebh            131421 non-null  int64         
 10  Radiation      131421 non-null  float64       
 11  GtiFixedTilt   131421 non-null  int64         
 12  GtiTracking    131421 non-null  int64         
 13  Precipitation  131421 non-null  float64       
 14  Humidity       131421 non-null  int32         
 15  

#### Display top 6 rows 

In [16]:
# Display top 6 rows
df.head(6)

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,GtiFixedTilt,GtiTracking,Precipitation,Humidity,SnowWater,Pressure,WindDirection,WindSpeed,Zenith,AlbedoDaily
0,2007-01-01 02:00:00,2007-01-01 01:00:00,PT60M,4.5,84,61.6,-1.0,0,0,0,...,0,0,22.1,67,0.0,1017.3,163.0,3.3,133,0.1
1,2007-01-01 03:00:00,2007-01-01 02:00:00,PT60M,4.6,72,61.8,-0.6,0,0,0,...,0,0,24.2,68,0.0,1016.7,160.0,3.8,144,0.1
2,2007-01-01 04:00:00,2007-01-01 03:00:00,PT60M,4.9,52,59.0,-0.2,0,0,0,...,0,0,25.5,69,0.0,1015.9,159.0,4.3,154,0.1
3,2007-01-01 05:00:00,2007-01-01 04:00:00,PT60M,5.4,20,76.9,0.2,0,0,0,...,0,0,26.2,69,0.0,1015.0,159.0,4.7,161,0.1
4,2007-01-01 06:00:00,2007-01-01 05:00:00,PT60M,5.8,-22,98.0,0.6,0,0,0,...,0,0,26.8,69,0.0,1014.2,159.0,5.0,161,0.1
5,2007-01-01 07:00:00,2007-01-01 06:00:00,PT60M,6.2,-54,97.5,1.4,0,0,0,...,0,0,27.6,71,0.0,1013.3,160.0,5.4,154,0.1


#### Create new features - year, month, and day

In [17]:
# Create new features(year, month, day) from start and end time of data collection  
df['YearPS'] = pd.to_datetime(df['PeriodStart']).dt.year
df['YearPE'] = pd.to_datetime(df['PeriodEnd']).dt.year

df['MonthPS'] = pd.to_datetime(df['PeriodStart']).dt.month
df['MonthPE'] = pd.to_datetime(df['PeriodEnd']).dt.month

df['HourPS'] = pd.to_datetime(df['PeriodStart']).dt.hour
df['HourPE'] = pd.to_datetime(df['PeriodEnd']).dt.hour

#### Display top 5 samples

In [18]:
# Display the top 5 samples
df.head(5)

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,WindDirection,WindSpeed,Zenith,AlbedoDaily,YearPS,YearPE,MonthPS,MonthPE,HourPS,HourPE
0,2007-01-01 02:00:00,2007-01-01 01:00:00,PT60M,4.5,84,61.6,-1.0,0,0,0,...,163.0,3.3,133,0.1,2007,2007,1,1,1,2
1,2007-01-01 03:00:00,2007-01-01 02:00:00,PT60M,4.6,72,61.8,-0.6,0,0,0,...,160.0,3.8,144,0.1,2007,2007,1,1,2,3
2,2007-01-01 04:00:00,2007-01-01 03:00:00,PT60M,4.9,52,59.0,-0.2,0,0,0,...,159.0,4.3,154,0.1,2007,2007,1,1,3,4
3,2007-01-01 05:00:00,2007-01-01 04:00:00,PT60M,5.4,20,76.9,0.2,0,0,0,...,159.0,4.7,161,0.1,2007,2007,1,1,4,5
4,2007-01-01 06:00:00,2007-01-01 05:00:00,PT60M,5.8,-22,98.0,0.6,0,0,0,...,159.0,5.0,161,0.1,2007,2007,1,1,5,6


#### Create a feature to identify time of sunrise

In [19]:
# Create a new feature for sunrise using the function below; fill time for sunrise
def time_to_sunrise(row):
    """
    A function for identifying time of sunrise.
    """
    value= 0
    if row['HourPS'] == 4 and row['HourPE'] == 5 and row['Radiation'] > 0:
        value= '05:00:00'
    elif row['HourPS'] == 5 and row['HourPE'] == 6 and row['Radiation'] > 0:
        value= '06:00:00'
    return value

#### Add the feature into the dataframe

In [20]:
# Create a new column using the function above
df['Sunrise'] = df.apply(time_to_sunrise, axis=1)
df.head(5)

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,WindSpeed,Zenith,AlbedoDaily,YearPS,YearPE,MonthPS,MonthPE,HourPS,HourPE,Sunrise
0,2007-01-01 02:00:00,2007-01-01 01:00:00,PT60M,4.5,84,61.6,-1.0,0,0,0,...,3.3,133,0.1,2007,2007,1,1,1,2,0
1,2007-01-01 03:00:00,2007-01-01 02:00:00,PT60M,4.6,72,61.8,-0.6,0,0,0,...,3.8,144,0.1,2007,2007,1,1,2,3,0
2,2007-01-01 04:00:00,2007-01-01 03:00:00,PT60M,4.9,52,59.0,-0.2,0,0,0,...,4.3,154,0.1,2007,2007,1,1,3,4,0
3,2007-01-01 05:00:00,2007-01-01 04:00:00,PT60M,5.4,20,76.9,0.2,0,0,0,...,4.7,161,0.1,2007,2007,1,1,4,5,0
4,2007-01-01 06:00:00,2007-01-01 05:00:00,PT60M,5.8,-22,98.0,0.6,0,0,0,...,5.0,161,0.1,2007,2007,1,1,5,6,0


#### Create a feature to identify time of sunset

In [21]:
# Create a new feature for sunset using the function below; fill time for sunset
def time_to_sunset(row):
    """
    A function for identifying time of sunset.
    """
    value= 0
    if row['HourPS'] == 17 and row['HourPE'] == 18 and row['Radiation']> 0:
        value= '18:00:00'
    elif row['HourPS'] == 18 and row['HourPE']== 19 and row['Radiation']> 0:
        value= '19:00:00'
    return value

#### Add the feature into dataframe

In [22]:
# Create a new column using the function above
df['Sunset'] = df.apply(time_to_sunset, axis=1)

#### Show unique values of sunset feature

In [23]:
# Unique values of sunset feature
df.Sunset.unique()

array([0, '18:00:00', '19:00:00'], dtype=object)

#### Summary of dataframe

In [24]:
# Show summary of our dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131421 entries, 0 to 131420
Data columns (total 29 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   PeriodEnd      131421 non-null  datetime64[ns]
 1   PeriodStart    131421 non-null  datetime64[ns]
 2   Period         131421 non-null  object        
 3   Temperature    131421 non-null  float64       
 4   Azimuth        131421 non-null  int64         
 5   CloudOpacity   131421 non-null  float64       
 6   DewpointTemp   131421 non-null  float64       
 7   DHI            131421 non-null  int64         
 8   DNI            131421 non-null  int64         
 9   Ebh            131421 non-null  int64         
 10  Radiation      131421 non-null  float64       
 11  GtiFixedTilt   131421 non-null  int64         
 12  GtiTracking    131421 non-null  int64         
 13  Precipitation  131421 non-null  float64       
 14  Humidity       131421 non-null  int32         
 15  

#### Change interger values to datetime object

In [25]:
# Changing integer values to datetime format
df_new = df.loc[~((df['Radiation'] == 0) & (df['DNI'] == 0) & (df['DHI'] == 0)),:]

#### Display summary of dataframe

In [ ]:
# Show summary of our dataframe
df_new.info()

#### Display 10 random samples

In [26]:
# display 10 random samples of the dataframe
df_new.sample(10)

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,Zenith,AlbedoDaily,YearPS,YearPE,MonthPS,MonthPE,HourPS,HourPE,Sunrise,Sunset
64887,2014-05-27 17:00:00,2014-05-27 16:00:00,PT60M,27.4,-165,40.0,14.2,549,7,6,...,21,0.11,2014,2014,5,5,16,17,0,0
46772,2012-05-02 22:00:00,2012-05-02 21:00:00,PT60M,12.0,91,68.4,8.8,106,0,0,...,65,0.11,2012,2012,5,5,21,22,0,0
13862,2008-07-31 16:00:00,2008-07-31 15:00:00,PT60M,28.5,-133,6.8,19.9,311,539,470,...,30,0.11,2008,2008,7,7,15,16,0,0
22846,2009-08-10 00:00:00,2009-08-09 23:00:00,PT60M,23.7,73,37.2,20.8,14,0,0,...,85,0.11,2009,2009,8,8,23,0,0,0
99037,2018-04-19 15:00:00,2018-04-19 14:00:00,PT60M,6.5,-123,90.2,4.3,68,0,0,...,44,0.10,2018,2018,4,4,14,15,0,0
94212,2017-09-30 14:00:00,2017-09-30 13:00:00,PT60M,14.9,-122,6.4,9.6,154,538,248,...,62,0.11,2017,2017,9,9,13,14,0,0
93738,2017-09-10 20:00:00,2017-09-10 19:00:00,PT60M,21.2,125,15.7,8.6,343,252,158,...,50,0.11,2017,2017,9,9,19,20,0,0
49476,2012-08-23 14:00:00,2012-08-23 13:00:00,PT60M,23.2,-108,29.2,18.2,329,61,40,...,54,0.11,2012,2012,8,8,13,14,0,0
117225,2020-05-16 11:00:00,2020-05-16 10:00:00,PT60M,17.9,-72,40.6,10.9,54,0,0,...,82,0.11,2020,2020,5,5,10,11,0,0
16766,2008-11-29 16:00:00,2008-11-29 15:00:00,PT60M,4.5,-161,61.5,-2.8,165,0,0,...,65,0.12,2008,2008,11,11,15,16,0,0


#### Convert datetime column

In [28]:
# Convert datetime column of endperiod to just date
df_new['Date'] = pd.to_datetime(df_new['PeriodEnd']).dt.normalize()

# Convert the column (it's a string) to datetime type
dt_series = pd.to_datetime(df_new['Date'])

# Create datetime index passing the datetime series
dt_index = pd.DatetimeIndex(dt_series.values)

# Set new index
df_new1 = df_new.set_index(dt_index)

C:\Users\sulma\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Display first 5 rows

In [29]:
# Display first 5 rows
df_new1.head()

,PeriodEnd,PeriodStart,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,...,AlbedoDaily,YearPS,YearPE,MonthPS,MonthPE,HourPS,HourPE,Sunrise,Sunset,Date
2007-01-01,2007-01-01 13:00:00,2007-01-01 12:00:00,PT60M,9.5,-122,86.2,8.6,2,0,0,...,0.1,2007,2007,1,1,12,13,0,0,2007-01-01
2007-01-01,2007-01-01 14:00:00,2007-01-01 13:00:00,PT60M,10.5,-133,90.6,9.8,11,0,0,...,0.1,2007,2007,1,1,13,14,0,0,2007-01-01
2007-01-01,2007-01-01 15:00:00,2007-01-01 14:00:00,PT60M,11.4,-145,89.1,11.0,26,0,0,...,0.1,2007,2007,1,1,14,15,0,0,2007-01-01
2007-01-01,2007-01-01 16:00:00,2007-01-01 15:00:00,PT60M,12.0,-158,85.3,11.7,49,0,0,...,0.1,2007,2007,1,1,15,16,0,0,2007-01-01
2007-01-01,2007-01-01 17:00:00,2007-01-01 16:00:00,PT60M,12.2,-173,84.2,12.0,61,0,0,...,0.1,2007,2007,1,1,16,17,0,0,2007-01-01


#### Summary of dataframe

In [30]:
# Display summary of dataframe
df_new1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71885 entries, 2007-01-01 to 2021-12-28
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   PeriodEnd      71885 non-null  datetime64[ns]
 1   PeriodStart    71885 non-null  datetime64[ns]
 2   Period         71885 non-null  object        
 3   Temperature    71885 non-null  float64       
 4   Azimuth        71885 non-null  int64         
 5   CloudOpacity   71885 non-null  float64       
 6   DewpointTemp   71885 non-null  float64       
 7   DHI            71885 non-null  int64         
 8   DNI            71885 non-null  int64         
 9   Ebh            71885 non-null  int64         
 10  Radiation      71885 non-null  float64       
 11  GtiFixedTilt   71885 non-null  int64         
 12  GtiTracking    71885 non-null  int64         
 13  Precipitation  71885 non-null  float64       
 14  Humidity       71885 non-null  int32         
 15  Sn

#### Tabulate the average for the solar and weather features

In [31]:
# Sample daily average for the solar and weather features
df_new1['Daily_radiation'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Radiation'].mean()
df_new1['Daily_DNI'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['DNI'].mean()
df_new1['Daily_DHI'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['DHI'].mean()
df_new1['Daily_Temp'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Temperature'].mean()
df_new1['Daily_Precip'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Precipitation'].mean()
df_new1['Daily_Humidity'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Humidity'].mean()
df_new1['Daily_Pressure'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Pressure'].mean()
df_new1['Daily_WindDir'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['WindDirection'].mean()
df_new1['Daily_WindSpeed'] = df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['WindSpeed'].mean()

#### Summary of dataframe

In [32]:
# display a summary of our dataframe
df_new1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71885 entries, 2007-01-01 to 2021-12-28
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PeriodEnd        71885 non-null  datetime64[ns]
 1   PeriodStart      71885 non-null  datetime64[ns]
 2   Period           71885 non-null  object        
 3   Temperature      71885 non-null  float64       
 4   Azimuth          71885 non-null  int64         
 5   CloudOpacity     71885 non-null  float64       
 6   DewpointTemp     71885 non-null  float64       
 7   DHI              71885 non-null  int64         
 8   DNI              71885 non-null  int64         
 9   Ebh              71885 non-null  int64         
 10  Radiation        71885 non-null  float64       
 11  GtiFixedTilt     71885 non-null  int64         
 12  GtiTracking      71885 non-null  int64         
 13  Precipitation    71885 non-null  float64       
 14  Humidity         7188

#### Drop irrelevant features

In [33]:
# Drop irrelevant features
new_df= df_new1.drop(['PeriodStart', 'YearPS',
                      'HourPS', 'HourPE',
                      'Sunrise', 'Sunset',
                      'MonthPS', 'YearPE'], axis= 1)

#### Display top 10 rows

In [34]:
# Display first 10 rows
new_df.head(10)

,PeriodEnd,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,Radiation,...,Date,Daily_radiation,Daily_DNI,Daily_DHI,Daily_Temp,Daily_Precip,Daily_Humidity,Daily_Pressure,Daily_WindDir,Daily_WindSpeed
2007-01-01,2007-01-01 13:00:00,PT60M,9.5,-122,86.2,8.6,2,0,0,2.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 14:00:00,PT60M,10.5,-133,90.6,9.8,11,0,0,11.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 15:00:00,PT60M,11.4,-145,89.1,11.0,26,0,0,26.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 16:00:00,PT60M,12.0,-158,85.3,11.7,49,0,0,49.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 17:00:00,PT60M,12.2,-173,84.2,12.0,61,0,0,61.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 18:00:00,PT60M,12.4,172,84.6,12.2,60,0,0,60.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 19:00:00,PT60M,12.4,157,86.6,12.2,45,0,0,45.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 20:00:00,PT60M,12.2,144,72.0,12.0,60,0,0,60.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 21:00:00,PT60M,12.0,132,47.7,11.9,59,0,0,59.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54
2007-01-01,2007-01-01 22:00:00,PT60M,11.7,122,37.2,11.4,9,0,0,9.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.9,210.4,4.54


#### Summary of dataframe

In [35]:
# Show a summary of the dataframe
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71885 entries, 2007-01-01 to 2021-12-28
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PeriodEnd        71885 non-null  datetime64[ns]
 1   Period           71885 non-null  object        
 2   Temperature      71885 non-null  float64       
 3   Azimuth          71885 non-null  int64         
 4   CloudOpacity     71885 non-null  float64       
 5   DewpointTemp     71885 non-null  float64       
 6   DHI              71885 non-null  int64         
 7   DNI              71885 non-null  int64         
 8   Ebh              71885 non-null  int64         
 9   Radiation        71885 non-null  float64       
 10  GtiFixedTilt     71885 non-null  int64         
 11  GtiTracking      71885 non-null  int64         
 12  Precipitation    71885 non-null  float64       
 13  Humidity         71885 non-null  int32         
 14  SnowWater        7188

#### Drop all duplicate values except for the last value

In [36]:
# Dropping ALL duplicate values exceept the last value
new_df = new_df[~new_df.Date.duplicated(keep = 'last')]

#### Display the number of rows and columns

In [37]:
# show the number of rows and columns available
new_df.shape

(5476, 31)

#### Top 10 samples

In [38]:
# Show top 10 samples
new_df.head(10)

,PeriodEnd,Period,Temperature,Azimuth,CloudOpacity,DewpointTemp,DHI,DNI,Ebh,Radiation,...,Date,Daily_radiation,Daily_DNI,Daily_DHI,Daily_Temp,Daily_Precip,Daily_Humidity,Daily_Pressure,Daily_WindDir,Daily_WindSpeed
2007-01-01,2007-01-01 22:00:00,PT60M,11.7,122,37.2,11.4,9,0,0,9.0,...,2007-01-01,38.2,0.0,38.2,11.63,31.83,97.0,1001.90,210.4,4.54
2007-01-02,2007-01-02 22:00:00,PT60M,6.0,122,0.0,-2.6,8,123,12,21.0,...,2007-01-02,260.2,662.3,40.9,6.00,5.54,57.4,1011.42,310.8,6.36
2007-01-03,2007-01-03 22:00:00,PT60M,8.4,122,0.0,2.1,9,109,11,21.0,...,2007-01-03,251.7,589.0,47.3,6.51,6.79,67.2,1017.44,225.7,5.16
2007-01-04,2007-01-04 22:00:00,PT60M,10.8,122,0.0,7.0,10,103,11,21.0,...,2007-01-04,211.8,354.3,88.9,9.18,10.27,81.0,1011.68,232.8,5.01
2007-01-05,2007-01-05 22:00:00,PT60M,14.3,122,92.7,13.8,2,0,0,2.0,...,2007-01-05,42.7,0.0,42.7,13.88,34.87,91.3,1004.53,218.8,5.80
2007-01-06,2007-01-06 22:00:00,PT60M,16.4,122,0.0,11.7,11,107,11,22.0,...,2007-01-06,230.3,477.1,64.6,16.78,25.72,89.4,996.20,242.4,7.02
2007-01-07,2007-01-07 22:00:00,PT60M,8.0,122,41.0,2.3,13,33,4,18.0,...,2007-01-07,255.3,593.1,52.9,7.90,9.57,62.3,1016.62,188.9,1.82
2007-01-08,2007-01-08 22:00:00,PT60M,7.6,122,0.3,3.9,13,122,12,25.0,...,2007-01-08,46.1,29.6,42.6,10.10,23.05,92.0,993.53,249.5,6.29
2007-01-09,2007-01-09 22:00:00,PT60M,4.6,122,0.8,-3.4,14,103,10,24.0,...,2007-01-09,199.7,242.0,127.0,3.91,6.93,53.0,1000.32,243.2,5.22
2007-01-10,2007-01-10 22:00:00,PT60M,0.4,122,0.0,-10.0,12,161,17,29.0,...,2007-01-10,253.9,533.6,69.8,0.65,4.19,51.5,1010.05,299.5,6.33


#### Display column names

In [39]:
# List out the column names
new_df.columns

Index(['PeriodEnd', 'Period', 'Temperature', 'Azimuth', 'CloudOpacity',
       'DewpointTemp', 'DHI', 'DNI', 'Ebh', 'Radiation', 'GtiFixedTilt',
       'GtiTracking', 'Precipitation', 'Humidity', 'SnowWater', 'Pressure',
       'WindDirection', 'WindSpeed', 'Zenith', 'AlbedoDaily', 'MonthPE',
       'Date', 'Daily_radiation', 'Daily_DNI', 'Daily_DHI', 'Daily_Temp',
       'Daily_Precip', 'Daily_Humidity', 'Daily_Pressure', 'Daily_WindDir',
       'Daily_WindSpeed'],
      dtype='object')

#### Select pertinent features

In [40]:
# Select column features
final_df= new_df[['MonthPE', 'Date',
                  'Daily_Temp','Daily_Precip',
                  'Daily_Humidity', 'Daily_Pressure',
                  'Daily_WindDir','Daily_WindSpeed',
                  'Daily_DNI', 'Daily_DHI',
                  'Daily_radiation']]

#### Top 5 rows

In [41]:
# Print the first 5 rows
final_df.head()

,MonthPE,Date,Daily_Temp,Daily_Precip,Daily_Humidity,Daily_Pressure,Daily_WindDir,Daily_WindSpeed,Daily_DNI,Daily_DHI,Daily_radiation
2007-01-01,1,2007-01-01,11.63,31.83,97.0,1001.90,210.4,4.54,0.0,38.2,38.2
2007-01-02,1,2007-01-02,6.00,5.54,57.4,1011.42,310.8,6.36,662.3,40.9,260.2
2007-01-03,1,2007-01-03,6.51,6.79,67.2,1017.44,225.7,5.16,589.0,47.3,251.7
2007-01-04,1,2007-01-04,9.18,10.27,81.0,1011.68,232.8,5.01,354.3,88.9,211.8
2007-01-05,1,2007-01-05,13.88,34.87,91.3,1004.53,218.8,5.80,0.0,42.7,42.7


#### Display the shape

In [42]:
# Show the number of rows and column
final_df.shape

(5476, 11)

#### Export cleaned dataset

In [43]:
# Save cleaned dataset
final_df.to_csv('../datasets/cleaned/cleaned_solar_irradiation.csv')